In [1]:
import os, logging

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
logging.getLogger("tensorflow").setLevel(logging.CRITICAL)
logging.getLogger("tensorflow_hub").setLevel(logging.CRITICAL)



In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
import pandas as pd
import os
import sys
import datetime
from pathlib import Path
import os
from absl import logging

sys.path.append('Enviroment')
from tf_trade_enviroment_v02 import MyTradeEnv

from tf_agents.environments import utils
from tf_agents.environments import tf_py_environment
from tf_agents.environments import wrappers

from tf_agents.agents.dqn import dqn_agent
from tf_agents.networks import q_network
from tf_agents.utils import common

from learningClass import learningHelper

logging.set_verbosity(logging.INFO)

tf.random.set_seed(12)
tf.print(tf.config.list_physical_devices('GPU') )
tf.compat.v1.enable_v2_behavior()

[]


## Read OCHLV data from the file

In [3]:
stocks_train = ['AAPL', 'C', 'AMD']
#stocks_train = ['AAPL']
stocks_test = ['MSFT']
history_range = {'start': datetime.datetime(2000, 1, 1), 
                 'end': datetime.datetime(2010, 10, 1)}

## Create and validate enviroment

In [5]:
my_history_length = 3
environment = wrappers.HistoryWrapper(MyTradeEnv(stocks_test, history_range), history_length=my_history_length)
#environment = MyTradeEnv(stocks_test, history_range)

#utils.validate_py_environment(environment, episodes=2)

print('action_spec:', environment.action_spec())
print('time_step_spec.observation:', environment.time_step_spec().observation)
print('time_step_spec.step_type:', environment.time_step_spec().step_type)
print('time_step_spec.discount:', environment.time_step_spec().discount)
print('time_step_spec.reward:', environment.time_step_spec().reward)



RemoteDataError: Unable to read URL: https://finance.yahoo.com/quote/MSFT/history?period1=946695600&period2=1285984799&interval=1d&frequency=1d&filter=history
Response Text:
b'<!DOCTYPE html>\n  <html lang="en-us"><head>\n  <meta http-equiv="content-type" content="text/html; charset=UTF-8">\n      <meta charset="utf-8">\n      <title>Yahoo</title>\n      <meta name="viewport" content="width=device-width,initial-scale=1,minimal-ui">\n      <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n      <style>\n  html {\n      height: 100%;\n  }\n  body {\n      background: #fafafc url(https://s.yimg.com/nn/img/sad-panda-201402200631.png) 50% 50%;\n      background-size: cover;\n      height: 100%;\n      text-align: center;\n      font: 300 18px "helvetica neue", helvetica, verdana, tahoma, arial, sans-serif;\n  }\n  table {\n      height: 100%;\n      width: 100%;\n      table-layout: fixed;\n      border-collapse: collapse;\n      border-spacing: 0;\n      border: none;\n  }\n 

In [5]:
utils.validate_py_environment(environment, episodes=2)

## Create traning and validation enviroment

In [6]:
train_env = tf_py_environment.TFPyEnvironment(MyTradeEnv(stocks_train, history_range))
eval_env = tf_py_environment.TFPyEnvironment(MyTradeEnv(stocks_test, history_range))
# train_env = tf_py_environment.TFPyEnvironment(wrappers.HistoryWrapper(MyTradeEnv(stocks_train, history_range), history_length=my_history_length))
# eval_env = tf_py_environment.TFPyEnvironment(wrappers.HistoryWrapper(MyTradeEnv(stocks_test, history_range), history_length=my_history_length))

## Define trainig paramters

In [7]:
learning_rate = 1e-3  

## Create q_network

In [8]:
#network configuration
fc_layer_params = (40,)

# as we are using dictionary in our enviroment, we will create preprocessing layer
preprocessing_layers = {
    'price': tf.keras.layers.Flatten(),
    'pos': tf.keras.layers.Flatten()
    }
preprocessing_combiner = tf.keras.layers.Concatenate(axis=-1)


#create a q_net
q_net = q_network.QNetwork(
    train_env.observation_spec(),
    train_env.action_spec(),
    preprocessing_layers=preprocessing_layers,
    preprocessing_combiner=preprocessing_combiner,
    fc_layer_params=fc_layer_params)

## Create the DQN-agent

In [9]:
#create optimizer
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)

#create a global step coubter
#train_step_counter = tf.Variable(0)
global_step = tf.compat.v1.train.get_or_create_global_step()

#create agent
agent = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    #train_step_counter=train_step_counter)
    train_step_counter=global_step)

agent.initialize()

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent.train = common.function(agent.train)

In [10]:
magent = learningHelper(train_env=train_env, test_env=eval_env, agent=agent, global_step=global_step, 
                        collect_episodes = 1,
                        collect_steps_per_iteration = 1000,
                        eval_interval=5,
                        replay_buffer_capacity=20000,
                        batch_size=500,
                        #collect_policy = mp,
                        log_interval = 10,
                        train_sequence_length = 1,
)
magent.restore_check_point()

INFO:absl:selected agent collect_policy
INFO:absl:No checkpoint available at ./checkpoint
INFO:absl:No checkpoint available at ./policy_checkpoint
INFO:absl:Initializing is done


In [11]:
%%time
history = magent.train_agent(11)
#magent.train_agent_with_avg_ret_condition(100, 10000, 100)

Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.
Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient inst

In [12]:
for _ in range(1):
    magent.evaluate_agent(100)


pnl: -2.9130302005011384 maxPnl: 3.0410304287639307
pnl: -0.5094072894240576 maxPnl: 8.099201261849478
pnl: 0.36058976126633135 maxPnl: 6.7533251564763495
pnl: -5.069367861190788 maxPnl: 0.18837078664020457
pnl: -5.485381227758948 maxPnl: 0.0
pnl: -6.541582974921312 maxPnl: 0.0
pnl: 5.102967449000444 maxPnl: 10.460109788517201
pnl: -5.339085289187186 maxPnl: 2.178990381689812
pnl: -4.756433064262204 maxPnl: 1.1089068267547701
pnl: -4.967018136995408 maxPnl: 1.0241429593371971
pnl: -0.9739641917452149 maxPnl: 4.242658266256553
pnl: -6.7127763719378 maxPnl: 0.31522633246553833
pnl: -3.3896907516858503 maxPnl: 1.8071478810711512
pnl: -6.401998670096065 maxPnl: 0.0
pnl: -4.655809815496618 maxPnl: 0.7718590093528479
pnl: 7.66491411293306 maxPnl: 12.899205869304664
pnl: -5.747698059043475 maxPnl: 0.0
pnl: -1.7347294057648588 maxPnl: 3.860729143442332
pnl: 3.6314820497520808 maxPnl: 9.826448592577037
pnl: 15.883939646321595 maxPnl: 21.54763239398877
pnl: -4.295136703400844 maxPnl: 1.010124593

In [37]:
magent.store_check_point()

In [16]:
magent.restore_check_point()